# Lecture 5: Simulation and Bootstrap

#### Zhentao Shi



<img src="graph/macau.jpg" width="1000">

In [ ]:
import numpy as np
import pandas as pd
import random

## Simulation


* check finite-sample performance
* bootstrap, a data-driven inference procedure
* generate non-standard distributions
* approximate integrals with no analytic expressions

## Appetizer

calculating $\pi$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(5, 5))
ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.add_patch(plt.Rectangle((-1,-1), 2, 2, fill=False))
ax.add_artist(plt.Circle((0, 0), 1, fill=False))
x = np.random.uniform(-1, 1, size=100)
y = np.random.uniform(-1, 1, size=100)
ax.scatter(x, y, s=25, c='blue', marker='o')
plt.show()


By laws of large numbers, $\pi$ can be approximated by a stochastic algorithm

$$
E\left[\boldsymbol{1}\left\{ x^{2}+y^{2}\leq1\right\} \right] = \frac{\pi r^{2}}{\left(2r\right)^{2}}= \frac{\pi}{4}
$$

it implies  $\pi=4\times E[ 1 \{  x^{2}+y^{2}\leq1 \}]$. 

In [ ]:
import numpy as np

n = 20000000
Z = 2 * np.random.rand(n, 2) - 1 # uniform distribution in [-1, 1]

inside = np.mean(np.sqrt(np.sum(Z**2, axis=1)) <= 1)
print(f"The estimated pi = {inside * 4:.6f}")


* Sample size can be made as large as the computer's memory permits.
* Iterate it with average of averages and so on, for higher accuracy.

## Writing Script

* A script is a piece of code for a particular purpose. 
* Thousands of lines are not written from the beginning to the end. 

* Recursive development
  * small manageable tasks
  * test code constantly 
  * encapsulate into DIY functions

* Integrate small pieces into the super structure. 
* Add comments to the script to facilitate readability
* Scprit (XXX.py) is more suitable than Jupyter notebook

## Finite Sample Evaluation


* Real world sample is finite
* Asymptotic theory is a mathematical apparatus to approximate finite sample distributions
* Modern econometric theory is built on asymptotics
* Simulation is one way to evaluate the accuracy of approximation.

## Example

For the OLS estimator, 

* Classical views $X$ as fixed regressions and only cares about the randomness of the error term.
* Modern econometrics textbook emphasizes that a random $X$ is more appropriate
for econometrics applications. 
* In rigorous textbooks, the moment of $X$ is explicitly stated as $E[X_i X_i'] < \infty$.

* A Pareto distribution with shape coefficient between 1 and 2 has finite population mean, but infinite variance. 
* If $X$ follows a
[Pareto distribution](https://en.wikipedia.org/wiki/Pareto_distribution) with shape coefficient 1.5, it violates the assumptions for OLS stated in most of econometric textbooks.
* Question: Is asymptotic inferential theory for the OLS estimator valid? 

We write a script to investigate this problem. The following steps develop the code.

 1. given a sample size, get the OLS `b_hat` and its associated `t_value`.
 2. wrap `t_value` as a user-defined function so that we can reuse it for many times.
 3. given a sample size, report the size under two distributions.
 4. wrap step 3 again as a user-defined function, ready for different sample sizes.
 5. develop the super structure to connect the workhorse functions.
 6. add comments and documentation.


### $t$-statistic

$$
\sqrt{n} (\hat{\beta} - \beta_0) |X = (X'X/n)^{-1}  (X' e /\sqrt{n}),
$$

the $k$-th element of the vector coefficient conditional on $X$ is
$$
\widehat{\beta}_{k}|X=\eta_{k}'\widehat{\beta}|X
\sim N\left(\beta_{k},\sigma^{2}\left(X'X\right)_{kk}^{-1}\right).
$$

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, t

def simulation(n, dist='normal', df=5):
    """
    Simulate the t-value of the second parameter in a linear regression model
    with n samples and the specified distribution of errors.
    """
    
    if dist == 'normal':
        e = np.random.normal(size=n)
    elif dist == 't':
        e = np.random.standard_t(df, size=n)

    b0 = np.array([1, 2])
    X = np.column_stack((np.ones(n), np.random.pareto(1.5, size=n)))
    # the above X is generated from a Pareto distribution
    Y = X.dot(b0) + e

    bhat = np.linalg.inv(X.T @ X) @ (X.T @ Y)
    bhat2 = bhat[1] # parameter we want to test

    e_hat = Y - X.dot(bhat)
    sigma_hat_square = np.sum(e_hat**2) / (n - 2)
    sig_B = np.linalg.inv(X.T.dot(X)) * sigma_hat_square
    t_value_2 = (bhat2 - b0[1]) / np.sqrt(sig_B[1, 1])

    return t_value_2



In [ ]:
from scipy.stats import t, norm

def report(n, Rep):
    # collect the test size from the two distributions
    # this function contains some repetitive code, but is OK for such a simple one
    TEST_SIZE = np.zeros(3)

    # e ~ normal distribution, under which the t-dist is exact
    # generate the t-statistics from the normal distribution
    Res = np.array([simulation(n, 'normal') for i in range(Rep)])
    
    TEST_SIZE[0] = np.mean(np.abs(Res) > t.ppf(0.975, n-2))
    TEST_SIZE[1] = np.mean(np.abs(Res) > norm.ppf(0.975))

    # e ~ t-distribution, under which the exact distribution is complicated.
    # we rely on asymptotic normal distribution for inference instead
    Res = np.array([simulation(n, 't', df) for i in range(Rep)])
    TEST_SIZE[2] = np.mean(np.abs(Res) > norm.ppf(0.975))

    return TEST_SIZE


In [ ]:
Rep = 1000
df = 1  # t dist. with df = 1 is Cauchy

# run the calculation of the empirical sizes for different sample sizes
NN = [5, 10, 200, 2000, 20000]

RES = pd.DataFrame(np.zeros((len(NN), 4)), columns=['n', 'exact', 'normal.asym', 'cauchy.asym'])

for i, n in enumerate(NN): # loop over different sample sizes
    # enumerate is a built-in Python function that returns an iterator containing the index and value of a list

    RES.iloc[i, 0] = n
    RES.iloc[i, 1:] = report(n, Rep)

print(RES)


### Simulation Results

* 1st column: the error is normal, use exact distribution theory to find the critical value (according to $t$-distribution.)

* 2nd column: still uses the normal error
  * change the critical value to asymptotic normal distribution
  
* 3rd column: error distribution is Cauchy
  * asymptotic approximation breaks down
  * test size does not converge to the nominal 5%  

### Observations

* $X$ is always Pareto 
* distribution of $X$ doesn't matter in our simulations

### Justification

* Self-normalized $t$ statistic does not break down despite that $X'X/n$ does not converge
* Regardless the distribution of $X$, when the error term is normal 
  * numerator follows $N(0,1)$
  * demonimator follows $\chi^2$

## World View about Uncertainty


* Fundamental question: how to quantify uncertainty.
  * data $(X_1,X_2,\ldots,X_n)$
  * sample mean $\bar{X}$
  * sample variance $s$
  * frequentist confidence interval about the population mean

* Asymptotics is imaginary.
* Let's be realistic: we have a finite sample $n$.

## Bootstrap

* Let $X_1, X_2, \ldots, X_n \sim F$ be an i.i.d. sample of $n$ observations following a distribution $F$. 
* The finite sample distribution of a statistic $T_n(\theta)\sim G_n(\cdot, F)$ usually depends on the sample size $n$, as well as the unknown true distribution $F$. 


### Key Idea

* Bootstrap replaces the unknown distribution $F$ in $G_n(\cdot, F)$ by the empirical distribution function

$$
\hat{F}_n(\cdot) = n^{-1} \sum_{i=1}^n 1\{\cdot \leq X_i\}
$$

* Bootstrap inference is drawn from the bootstrap distribution

$$
G_n(\cdot, \hat{F}_n)
$$


### Compare to Asymptotic Theory

* Bootstrap is a finite-sample practice 
* It doesn't refer to an imaginary world where $n\to \infty$ at its face value


* Asymptotic theory approximates $G_n(\cdot, F)$ by its limit 

$$G(\cdot, F) := \lim_{n\to\infty} G_n(\cdot, F).$$ 

* In many cases $G(\cdot, F)$ is independent of $F$ and it becomes $G(\cdot)$. Such a $T_n(\theta)$ is called *asymptotically pivotal*, 

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF

x = np.random.uniform(size=10)
ecdf = ECDF(x)

# get unique sorted values of x and corresponding values of y
ux, uy = np.unique(ecdf.x, return_index=False, return_inverse=False, return_counts=False), ecdf.y[np.unique(ecdf.y, return_index=True, return_inverse=False, return_counts=False)[1]]

# add last point (1,1) to the step function
ux = np.concatenate((ux, [1]))
uy = np.concatenate((uy, [1]))

# create step function with horizontal lines
plt.plot(ux[:-1], uy[:-1], drawstyle='steps-pre')
plt.plot([ux[-2], ux[-1]], [uy[-2], uy[-2]], color='C0', linestyle='--')
plt.plot([ux[-1], ux[-1]], [uy[-2], uy[-1]], color='C0', linestyle='--')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('ECDF for uniform distribution')
plt.show()

### Nonparametric Bootstrap

* Implementation of bootstrap is a simulation exercise. 
* In an i.i.d. environment, $n$ observations are drawn with equal weight and **with replacement** from the realized sample

### Variants of Bootstrap Schemes

* Block bootstrap: preserve dependence structure
  * dependent dataset such as time series
  * clustering data or networks

* parametric bootstrap
  * In regressions we fix the regressors $X$

In [ ]:
import pandas as pd
import numpy as np
n = 9  # real sample size
d0 = pd.DataFrame({'x': np.random.normal(size=n)})
print(d0)

# bootstrap
boot_Rep = 3  # bootstrap 3 times
d_boot = np.zeros((n, boot_Rep))  # save the bootstrap sample
d_boot_index = np.zeros((n, boot_Rep), dtype=int)  # save the bootstrap sample index
for b in range(boot_Rep):
    boot_index = np.random.choice(d0.index, n, replace=True) # sampling with replacement
    d_boot[:, b] = d0['x'].iloc[boot_index]
    d_boot_index[:, b] = boot_index

d_boot_df = pd.DataFrame(d_boot)  # convert the bootstrap samples to DataFrame
print(d_boot_df)
print(d_boot_index)


**Example**: In one sample, there is only one sample mean. How to compute the variance of the sample mean? 

In [ ]:
np.var(d0['x'])/n # the variance of the sample mean

What if you forget the formula?

### Bootstrap Estimation

* Bootstrap is nothing but a loop.
* Bootstrap is convenient. 
  * Analytic formula of the variance of an econometric estimator can be complex to derive or code up.

In [ ]:
boot_Rep = 1000  # bootstrap many times times
d_boot = np.zeros((n, boot_Rep))  # save the bootstrap sample 
for b in range(boot_Rep):
    boot_index = np.random.choice(n, n, replace=True)
    d_boot[:, b] = d0['x'][boot_index]
 
np.mean(d_boot, axis=0).var()  # standard deviation of the bootstrap means across boot samples

### Bootstrap Test

* Bootstrap is particularly helpful in inference

### Example

* Test a hypothesis about the population mean. 

* Use $t$-statistic
* Distribution of the sample is either
  * normal
  * zero-centered chi-square 

* We will show that the bootstrap test size is
more precise than that of the asymptotic approximation.

In [ ]:
import numpy as np

def T_stat(Y, mu):
    n = len(Y)
    return np.sqrt(n) * (np.mean(Y) - mu) / np.std(Y, ddof=1)

def boot_test(Y, alpha, boot_Rep):
    n = len(Y)
    boot_T = []

    for r in range(boot_Rep):
        
        resampled_Y = Y[np.random.choice(n, n, replace=True)]
        boot_T.append(abs(T_stat(resampled_Y, np.mean(Y))))
        # this is a key line
        # `mu` must be replaced as mean(Y)

    boot_critical_value = np.quantile(boot_T, 1 - alpha)
    return boot_critical_value



### Bootstrap World

* The null hypothesis must be imposed no matter the hypothesized parameter is true value or not.

* Bootstrap $t$-statistic is

$$
T^{*}_{n} = \frac{\sqrt{n} (\bar{X^{*}} - \bar{X}) } { s^{*}  }.
$$



### Design the Statistic


* In the bootstrap world the **true** distribution is $F_n$
* The bootstrap $t$-statistic is centered around $\bar{X}$, the sample mean of $F_n$
* If we wrongly center it around the population mean $\theta$, then the test will have no power when the null hypothesis is false.

The following chuck of code report the rejection probability from three decision rules.

In [ ]:
from scipy.stats import t, norm, chi2

def compare():
    if distribution == "normal":
        X = np.random.normal(size=n)
    elif distribution == "chisq":
        X = chi2.rvs(n-1, size=n) - (n - 1)

    t_value_X = T_stat(X, mu)

    exact = abs(t_value_X) > t.ppf(1 - alpha/2, n-1)
    asym = abs(t_value_X) > norm.ppf(1 - alpha/2)
    boot_rule = abs(t_value_X) > boot_test(X, alpha, boot_Rep)

    return np.array([exact, asym, boot_rule])

In [ ]:
n = 10
distribution = "normal"
boot_Rep = 199
MC_rep = 2000 # number of Monte Carlo replications
alpha = 0.05
mu = 0

res = pd.DataFrame(index = range(MC_rep), columns=['exact', 'asym', 'boot'])

for i in range(MC_rep):
    res.iloc[i,:] = compare()

res.mean()


### Results

* The program reports the empirical size.
* Nominal size of the test is 5%. 
* Bootstrap test is more accurate


* When the underlying distribution is a $\chi^2$, the exact distribution is difficult 
to derive analytically. 
* However, we can still compare the asymptotic size with the bootstrap size.

In [ ]:
distribution = "chisq"
boot_Rep = 199
MC_rep = 2000
alpha = 0.05
mu = 0

for i in range(MC_rep):
    res.iloc[i,:] = compare()

res.mean()

* Here the "exact test" is no longer exact. 
* The asymptotic test works fairly reasonable
* Bootstrap is closer to the nominal size 5%.